# -----------------------第四版本-----------------------
###   使用两种算法进行修正标准覆盖半径
###   1-站高-按照覆盖传播模型进行修正
####   第一次修正后距离 = 实际站高覆盖距离/30米站高的覆盖距离*分场景和制式的标准覆盖距离
###   2-周边楼宇高度-按照周边建筑高度修正
####   覆盖区域建筑物平均高度=(∑（50%覆盖半径内建筑占地面积*建筑高度）)/(∑（50%覆盖半径内建筑占地面积）)
####   高度差比值 = 高度差 / 基站高度 按照区间进行修正
## 去掉室分和高铁，添加微站逻辑站点

In [1]:
import numpy as np
import pandas as pd
import mytools
import geopandas as gpd

In [2]:
#导入工参数据
gongcan = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/20200712第四版本/1-基础数据/ltejizhan20200616&微站_合并.csv',encoding='gbk'),usecols=[
    '小区CGI(*)','所属地区','小区中文名','RRU经度', 'RRU纬度','天线挂高','工作频段', '方位角','归属区县','厂家名称' ])
#设置变量通用名
print('设置变量通用名')
id = '小区CGI(*)'
lon = 'RRU经度'
lat = 'RRU纬度'
zhishi = '工作频段'
fw = '方位角'

gongcan = gongcan.loc[(gongcan['RRU经度']>108)&
            (gongcan['RRU经度']<117)&
            (gongcan['RRU纬度']>28)&
            (gongcan['RRU纬度']<34)&
           (gongcan['方位角'].notnull())]

设置变量通用名


In [6]:
#只要宏站
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('W-')]
gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('R')]
# gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('M-')]

In [12]:
3334+463

3797

In [11]:
369741+6265-40831-3313-3334-328065

463

In [5]:
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
gongcan.loc[gongcan['小区中文名'].str.contains('M-')].shape

(18098, 10)

In [ ]:
gongcan.shape

In [ ]:
#打标场景
gongcan = mytools.gisn.points_coverage_merge(gongcan,lon,lat)
print('添加覆盖距离')
gongcan_loc_juli = mytools.gisn.changjing_distance(gongcan,'区域类型',zhishi)

In [ ]:
gongcan_loc_juli.head(1)

## 按照覆盖模型修复

In [ ]:
gongcan_loc_juli['工作频段'] = gongcan_loc_juli['工作频段'].apply(lambda x: 900 if x=='FDD900' else 1800)

In [ ]:
gd,band= map(np.log10, (gongcan_loc_juli['天线挂高'],gongcan_loc_juli['工作频段']))

In [ ]:
gongcan_loc_juli['覆盖模型距离'] = np.power(10,
                                      (148 - (46.3+33.9*band-13.82*gd-(-3.81))- 3)/(44.9-6.55*gd))*1000

In [ ]:
gongcan_loc_juli['30米覆盖模型距离'] = np.power(10,
                                      (148 - (46.3+33.9*band-13.82*np.log10(30)-(-3.81))- 3)/(44.9-6.55*np.log10(30)))*1000

In [ ]:
gongcan_loc_juli['使用覆盖半径'] = gongcan_loc_juli['覆盖模型距离'] /gongcan_loc_juli['30米覆盖模型距离']*gongcan_loc_juli['distance']

In [ ]:
def fuck(x):
    if x['区域类型']=='主城区':
        if x['工作频段']==900:
            if x['使用覆盖半径']<200:
                return 200
            elif x['使用覆盖半径']>400:
                return 400
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<150:
                return 150
            elif x['使用覆盖半径']>350:
                return 350
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='县城':
        if x['工作频段']==900:
            if x['使用覆盖半径']<300:
                return 300
            elif x['使用覆盖半径']>430:
                return 430
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<250:
                return 250
            elif x['使用覆盖半径']>400:
                return 400
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='一般城区':
        if x['工作频段']==900:
            if x['使用覆盖半径']<400:
                return 400
            elif x['使用覆盖半径']>1200:
                return 1200
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<350:
                return 350
            elif x['使用覆盖半径']>800:
                return 800
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='乡镇':
        if x['工作频段']==900:
            if x['使用覆盖半径']<430:
                return 430
            elif x['使用覆盖半径']>2200:
                return 2200
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<400:
                return 400
            elif x['使用覆盖半径']>1000:
                return 1000
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='农村':
        if x['工作频段']==900:
            if x['使用覆盖半径']<1200:
                return 1200
            elif x['使用覆盖半径']>2500:
                return 2500
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<800:
                return 800
            elif x['使用覆盖半径']>1500:
                return 1500
            else:
                return x['使用覆盖半径']
    else:
        return 0

In [ ]:
gongcan_loc_juli['distance_use'] = gongcan_loc_juli.apply(fuck,axis=1)

In [ ]:
gongcan_loc_juli.to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/1_修正半径_1覆盖模型修正.csv',encoding='gbk')

## 按照周边建筑高度修复

In [ ]:
gongcan_loc_juli.head(1)

In [ ]:
gongcan_loc_juli['distance_yb'] = gongcan_loc_juli['distance']/2

In [ ]:
gongcan_sectors = mytools.gisn.add_sectors_df(gongcan_loc_juli,coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'distance_yb'],
    has_z=False,sec_col='geometry',shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
f = mytools.othern.file_name_paths('G:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/',find='tab')

In [ ]:
sjoin_data = gpd.GeoDataFrame()
for name in f:
    tuceng_t = gpd.read_file(name,encoding = 'gbk')
    tuceng_t = tuceng_t.to_crs({'init': 'epsg:4326'})
    linshi_t = gpd.sjoin(gongcan_sectors,tuceng_t)
    linshi_t = linshi_t[[ '小区CGI(*)','建筑物高度','SHAPE_Area']]
    sjoin_data = sjoin_data.append(linshi_t)
    print(name,'ok')

In [ ]:
sjoin_data['volume'] = sjoin_data['建筑物高度'] * sjoin_data['SHAPE_Area']

In [ ]:
sjoin_data_volume_h = sjoin_data.groupby('小区CGI(*)').agg({'volume':'sum','SHAPE_Area':'sum'})

In [ ]:
sjoin_data_volume_h['h_use'] = sjoin_data_volume_h['volume'] / sjoin_data_volume_h['SHAPE_Area']

In [ ]:
sjoin_data_linshi = sjoin_data_volume_h.reset_index()

In [ ]:
gongcan_zhangao = gongcan_loc_juli.merge(sjoin_data_linshi[['小区CGI(*)','h_use']],how='left',on='小区CGI(*)')

In [ ]:
gongcan_zhangao['高度建筑比'] = (gongcan_zhangao['天线挂高']-gongcan_zhangao['h_use'])/gongcan_zhangao['天线挂高']

In [ ]:
def fuck2(x):
    if (pd.isna(x['distance_use'])) | (x['distance_use']==0 ):
        return x['distance']
    elif pd.isna(x['高度建筑比']):
        return x['distance_use']
    elif x['高度建筑比']>=0.4:
        return x['distance_use']*1.1
    elif x['高度建筑比']>=0.2 and x['高度建筑比']<0.4:
        return x['distance_use']
    elif x['高度建筑比']>=0 and x['高度建筑比']<0.2:
        return x['distance_use']*0.8
    elif x['高度建筑比']<0:
        return x['distance_use']*0.5
    else:
        return x['distance_use']

In [ ]:
gongcan_zhangao['distance_use_3'] = gongcan_zhangao.apply(fuck2,axis=1)

In [ ]:
gongcan_zhangao.loc[gongcan_zhangao['distance_use_3'].isna()]

In [ ]:
gongcan_zhangao.loc[gongcan_zhangao['小区中文名'].str.contains('M-'),'distance_use_3']=50

In [ ]:
gongcan_zhangao.to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/1_修正半径_2周边建筑修正.csv',encoding='gbk')

In [ ]:
mytools.sound()